In [ ]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *

In [ ]:
%matplotlib notebook
import os
import sys
import glob
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import scipy
from skimage.transform import *
plt.ioff()

from mpl_toolkits.mplot3d import Axes3D

import cv2


In [ ]:
class drawing:
    def init(self):
        glutInit(sys.argv)
    def loadImage(self,fileName):
        im = cv2.imread(fileName)
        return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    def loadImageColor(self,fileName):
        def load_image(fname):
            return np.asarray(Image.open(fname))
        return load_image(fileName)
    def showImage(self,image, title="fig", small=False):
        if small:
            plt.figure(figsize=(2,3))
        else:
            fig = plt.figure()    
        plt.title(title)
        plt.imshow(image, cmap='gray')
        plt.show()
    def showImageColor(self,image, title, small=False):
        if small:
            plt.figure(figsize=(2,3))
        else:
            fig = plt.figure()    
        plt.title(title)
        plt.imshow(image)
        plt.show()
    def transform_rectangle(self,img,pts1,pts2):
        M = cv2.getPerspectiveTransform(pts1,pts2)
        dst = cv2.warpPerspective(img,M,(300,650))

        #plt.subplot(121),plt.imshow(img),plt.title('Input')
        #plt.subplot(122),plt.imshow(dst),plt.title('Output')
        #plt.show()
        return dst
    def transform_net(self,img,pts1,pts2):
        M = cv2.getPerspectiveTransform(pts1,pts2)
        dst = cv2.warpPerspective(img,M,(300,27))

        #plt.subplot(121),plt.imshow(img),plt.title('Input')
        #plt.subplot(122),plt.imshow(dst),plt.title('Output')
        #plt.show()
        return dst
    def get_rectangle(self,img,height,width):
        return img[:height,:width]

    def loadTexture(self,img):
        #img = pygame.image.load("./images/tennis-court-3d-model_D.jpg")
        #print(type(img))
        #textureData = pygame.image.tostring(img, "RGB", 1)
        #width = img.get_width()
        #height = img.get_height()
        #print(img)
        width = img.shape[1]
        height = img.shape[0]
        textureData = img
        #textureData = np.random.randint(256,size=(height, width, 3))
        
        bgImgGL = glGenTextures(1)
        glBindTexture(GL_TEXTURE_2D, bgImgGL)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
        glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, textureData)
        glEnable(GL_TEXTURE_2D)
        return bgImgGL

    def Plane(self,texture_id,pts,color=[0,0,0,0]):
        glEnable(GL_TEXTURE_2D)
        #print(texture_id)
        glBindTexture(GL_TEXTURE_2D, texture_id)
        glBegin(GL_QUADS)
        #glColor4f(color[0],color[1],color[2],color[3]);
        glTexCoord2f(0, 0)
        glVertex3f(pts[0][0], pts[0][1], pts[0][2])
        glTexCoord2f(1, 0)
        glVertex3f(pts[1][0], pts[1][1],  pts[1][2])
        glTexCoord2f(1, 1)
        glVertex3f(pts[2][0], pts[2][1], pts[2][2])
        glTexCoord2f(0, 1)
        glVertex3f(pts[3][0], pts[3][1],  pts[3][2])
        glEnd()
    def main(self,method=0):
        if method==0: #tennis manual
            fileName1 = './images/tennis-court-3d-model_D.jpg'
            img = self.loadImage(fileName1)
            #showImageColor = showImageColor(img,'image')
            court_pts = np.float32([[-0.3,-0.6,0],[0.3,-0.6,0],[0.3,0.6,0],[-0.3,0.6,0]])
            net_pts = np.float32([[0.3,0,0.1],[-0.3,0,0.1],[-0.3,0,0],[0.3,0,0]])
            pygame.init()
            display = (1280,800)
            pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
            gluPerspective(20, (display[0]/display[1]), 0, 50.0)
            glTranslatef(0.0,0.0, -5)
            glRotatef(120, 0, 0, 1)
            glRotatef(45, 0, 1, 0)
            glRotatef(30, 1, 0, 0)

            s_pts1 = np.float32([[99,115],[226,97],[212,220],[391,177]])
            s_pts2 = np.float32([[0,0],[300,0],[0,650],[300,650]])
            #138,140 294,116 291,130 141,155
            n_pts1 = np.float32([[138,140],[294,116],[141,155],[291,130]])
            n_pts2 = np.float32([[0,0],[300,0],[0,650],[300,650]])

            t = self.transform_rectangle(img,s_pts1,s_pts2)
            n = self.transform_rectangle(img,n_pts1,n_pts2)

            t1 = self.loadTexture(t)
            t2 = self.loadTexture(n)
            #self.loadTexture()
            done = False
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        #quit()

                glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
                
                self.Plane(t1,court_pts)
                self.Plane(t2,net_pts)
                pygame.display.flip()
        elif method==1: #bus manual
            fileName1 = './images/bus.png'

            img = self.loadImage(fileName1)
            #self.showImage(img)
            #showImageColor = showImageColor(img,'image')
            roof_pts = np.float32([[-0.3,-0.9,0],[0.3,-0.9,0],[0.3,0.9,0],[-0.3,0.9,0]])
            front_pts = np.float32([[0.3,0.9,-0.6],[-0.3,0.9,-0.6],[-0.3,0.9,0],[0.3,0.9,0]])
            side_pts = np.float32([[-0.3,0.9,-0.6],[-0.3,-0.9,-0.6],[-0.3,-0.9,0],[-0.3,0.9,0]])
            # x positive left top
            # y positive left down
            
            

            s_pts1 = np.float32([[337,97],[283,95],[120,142],[181,159]])
            s_pts2 = np.float32([[0,0],[300,0],[300,650],[0,650]])
            #138,140 294,116 291,130 141,155
            n_pts1 = np.float32([[115,217],[181,228],[181,159],[120,142]])
            n_pts2 = np.float32([[0,0],[300,0],[300,650],[0,650]])
            
            r_pts1 = np.float32([[181,228],[329,160],[337,97],[181,159]])
            r_pts2 = np.float32([[0,0],[300,0],[300,650],[0,650]])

            t = self.transform_rectangle(img,s_pts1,s_pts2)
            #self.showImage(t)
            n = self.transform_rectangle(img,n_pts1,n_pts2)
            #self.showImage(n)
            r = self.transform_rectangle(img,r_pts1,r_pts2)
            #self.showImage(r)
            
            pygame.init()
            display = (1280,800)
            pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
            gluPerspective(20, (display[0]/display[1]), 0, 50.0)
            glTranslatef(0.0, 0.0, -5)
            glRotatef(120, 0, 0, 1)
            glRotatef(45, 0, 1, 0)
            glRotatef(30, 1, 0, 0)
            t1 = self.loadTexture(t)
            t2 = self.loadTexture(n)
            t3 = self.loadTexture(r)
            #glEnable(GL_DEPTH_TEST)
            glDepthMask(GL_FALSE)
            glDepthFunc(GL_LESS)
            done = False
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        #quit()
                glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
                self.Plane(t2,front_pts)
                self.Plane(t3,side_pts)
                self.Plane(t1,roof_pts)
                pygame.display.flip()
                glRotatef(1, 0, 0, 1)
                pygame.time.wait(10)
        elif method==2: #tennis cropped
            fileName1 = './tennis_xy_clean.png'
            fileName2 = './tennis_xz.png'
            fileName3 = './tennis_person_resize.png'
            img1 = self.loadImage(fileName1)
            img2 = self.loadImage(fileName2)
            img3 = cv2.flip(self.loadImage(fileName3),1)
            #self.showImage(img1)
            mw = 0.4
            ml = mw/6*13
            mh = mw/12*2
            hs = 0.08
            
            
            court_pts = np.float32([[-mw*3/2,-ml*4/3,0],[mw*3/2,-ml*4/3,0],[mw*3/2,ml*4/3,0],[-mw*3/2,ml*4/3,0]])
            net_pts = np.float32([[mw-0.02,-0.03,mh],[-mw+0.02,-0.03,mh],[-mw+0.02,-0.03,0],[mw-0.02,-0.03,0]])
            human_pts = np.float32([[-0.07,-ml-mh/2+hs,mh*3],[-0.07,-ml+mh/2+hs,mh*3],[-0.07,-ml+mh/2+hs,0],[-0.07,-ml-mh/2+hs,0]])
            pygame.init()
            display = (1280,800)
            pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
            gluPerspective(20, (display[0]/display[1]), 0, 50.0)
            glTranslatef(0.0,0.0, -5)
            glRotatef(120, 0, 0, 1)
            glRotatef(45, 0, 1, 0)
            glRotatef(30, 1, 0, 0)
            
            glRotatef(-80, 0, 0, 1)
            
            t_height = img1.shape[0]
            t_width = img1.shape[1]
            n_height = img2.shape[0]
            n_width = img2.shape[1]
            h_height = img3.shape[0]
            h_width = img3.shape[1]

            t = self.get_rectangle(img1,t_height,t_width)
            n = self.get_rectangle(img2,n_height,n_width)
            h = self.get_rectangle(img3,h_height,h_width)

            t1 = self.loadTexture(t)
            t2 = self.loadTexture(n)
            t3 = self.loadTexture(h)
            
            done = False
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
                
                self.Plane(t1,court_pts,[0,0.5,1,1])
                self.Plane(t2,net_pts,[0,1,0,1])
                self.Plane(t3,human_pts,[1,0,0,1])
                pygame.display.flip()
                pygame.time.wait(10)
                #glRotatef(1, 0, 0, 1)
                
        elif method==3: #bus cropped
            fileName1 = './bus_xy.png'
            fileName2 = './bus_yz_resize.png'
            fileName3 = './bus_xz_resize.png'
            img1 = self.loadImage(fileName1) #roof
            img2 = self.loadImage(fileName2) #front
            img3 = self.loadImage(fileName3) #side

            
            
            #self.showImage(img1)
            #self.showImage(img2)
            #self.showImage(img3)
            #showImageColor = showImageColor(img,'image')
            roof_pts = np.float32([[-0.3,-0.9,0],[0.3,-0.9,0],[0.3,0.9,0],[-0.3,0.9,0]])
            front_pts = np.float32([[0.3,0.9,-0.6],[-0.3,0.9,-0.6],[-0.3,0.9,0],[0.3,0.9,0]])
            side_pts = np.float32([[-0.3,0.9,-0.6],[-0.3,-0.9,-0.6],[-0.3,-0.9,0],[-0.3,0.9,0]])
            # x positive left top
            # y positive left down
            
            r_height = img1.shape[0]
            r_width = img1.shape[1]
            f_height = img2.shape[0]
            f_width = img2.shape[1]
            s_height = img3.shape[0]
            s_width = img3.shape[1]

            r = self.get_rectangle(img1,r_height,r_width)
            f = self.get_rectangle(img2,f_height,f_width)
            s = self.get_rectangle(img3,s_height,s_width)
            #self.showImage(s)
            pygame.init()
            display = (1280,800)
            pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
            gluPerspective(30, (display[0]/display[1]), 0, 50.0)
            glTranslatef(0.0, 0.0, -5)
            glRotatef(120, 0, 0, 1)
            glRotatef(45, 0, 1, 0)
            glRotatef(30, 1, 0, 0)
            
            glRotatef(50, 0, 0, 1)
            
            t1 = self.loadTexture(r)
            t2 = self.loadTexture(f)
            t3 = self.loadTexture(s)
            #glEnable(GL_DEPTH_TEST)
            glDepthMask(GL_FALSE)
            glDepthFunc(GL_LESS)
            done = False
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        #quit()
                glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
                self.Plane(t2,front_pts,[0,0.5,1,1])
                self.Plane(t3,side_pts,[1,0.5,0,1])
                self.Plane(t1,roof_pts,[0.5,0,0.5,1])
                pygame.display.flip()
                #glRotatef(1, 0, 0, 1)
                pygame.time.wait(10)

In [ ]:
    
thing = drawing()
thing.main(2)